In [1]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import pandas as pd
import json
import time
import math
import random

import data_module as dm

from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
train_dl, test_dl, train_len, test_len = dm.get_pred_fst()

In [14]:
it = iter(train_dl)
fst = next(it)
fst = fst[0]

In [15]:
print(fst)

tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 1,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])


In [48]:
class FstPred(nn.Module):
    def __init__(self):
        super(FstPred, self).__init__()
        
        self.fc1 = nn.Linear(594, 1024)
        self.fc3 = nn.Linear(1024, 1500)
        self.sm = nn.Softmax(1)
        
    def forward(self, input):
        x = F.relu(self.fc1(input.to(torch.float)))
        x = F.relu(self.fc3(x))
        x = self.sm(x)
        
        return x

In [49]:
# Trainer helper functions from 
# https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s' % (asMinutes(s))

In [50]:
def train(model, input, target, optimizer, criterion):
    optimizer.zero_grad()
    
    input = input.to(device)
    out = model(input)
    
    loss = criterion(out, target.squeeze(-1).to(torch.int64))
    
    loss.backward()
    optimizer.step()
    
    return loss
    
def train_epoch(model, n_epochs, print_every = 50, lr=0.003):
    start = time.time()
    print_loss_total = 0
    
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    for epoch in range(n_epochs):
        for i, data in enumerate(train_dl):

            input = data[0].to(device) # back to data
            target = data[1].to(device) # back to data

            loss = train(model, input, target, optimizer, criterion)
            print_loss_total += loss

            if (i % print_every == 0) and (i != 0): # Change back to i
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0



                print('Epoch: %d Elapsed: %s Percent of epoch Complete: (%d%%) %.4f' % (epoch, timeSince(start, i / (train_len / 128)),
                                                                  i / (train_len / 32) * 100, print_loss_avg))


In [ ]:
model = FstPred().to(device)

train_epoch(model, 5, 500)

Epoch: 0 Elapsed: 0m 1s Percent of epoch Complete: (13%) 7.3137
Epoch: 0 Elapsed: 0m 3s Percent of epoch Complete: (26%) 7.2942
Epoch: 0 Elapsed: 0m 4s Percent of epoch Complete: (39%) 7.2950
Epoch: 0 Elapsed: 0m 5s Percent of epoch Complete: (52%) 7.2957
Epoch: 0 Elapsed: 0m 7s Percent of epoch Complete: (65%) 7.2955
Epoch: 0 Elapsed: 0m 9s Percent of epoch Complete: (78%) 7.2945
Epoch: 0 Elapsed: 0m 10s Percent of epoch Complete: (91%) 7.2944
Epoch: 1 Elapsed: 0m 13s Percent of epoch Complete: (13%) 12.1373
Epoch: 1 Elapsed: 0m 15s Percent of epoch Complete: (26%) 7.2951
Epoch: 1 Elapsed: 0m 16s Percent of epoch Complete: (39%) 7.2955
Epoch: 1 Elapsed: 0m 18s Percent of epoch Complete: (52%) 7.2925
Epoch: 1 Elapsed: 0m 19s Percent of epoch Complete: (65%) 7.2936
Epoch: 1 Elapsed: 0m 21s Percent of epoch Complete: (78%) 7.2946
Epoch: 1 Elapsed: 0m 22s Percent of epoch Complete: (91%) 7.2946
Epoch: 2 Elapsed: 0m 25s Percent of epoch Complete: (13%) 12.1354
Epoch: 2 Elapsed: 0m 27s Perc